## Deliverable 2. Create a Customer Travel Destinations Map.

In [16]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [17]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khuzhir,RU,53.19,107.34,51.55,74,55,3.27,broken clouds
1,1,Ushuaia,AR,-54.80,-68.30,38.86,41,75,4.61,broken clouds
2,2,Shingu,JP,33.73,135.98,77.92,89,100,6.53,overcast clouds
3,3,Biscarrosse,FR,44.39,-1.17,76.32,60,0,13.80,clear sky
4,4,Lebu,CL,-37.62,-73.65,54.23,67,1,9.51,clear sky


In [18]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 75


In [19]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Khuzhir,RU,53.19,107.34,51.55,74,55,3.27,broken clouds
4,4,Lebu,CL,-37.62,-73.65,54.23,67,1,9.51,clear sky
6,6,Rikitea,PF,-23.12,-134.97,72.88,77,6,12.75,light rain
13,13,Emba,KZ,48.83,58.14,61.72,27,16,8.14,few clouds
14,14,Albany,US,42.60,-73.97,71.28,73,45,4.32,scattered clouds
18,18,Hilo,US,19.73,-155.09,73.40,90,75,5.75,broken clouds
20,20,Veinticinco De Mayo,AR,-35.43,-60.17,68.13,45,0,8.48,clear sky
22,22,Port Alfred,ZA,-33.59,26.89,59.52,70,68,19.69,light rain
23,23,Chaumont,FR,48.12,5.13,70.38,67,39,7.27,scattered clouds
26,26,Chuy,UY,-33.70,-53.46,65.03,53,0,1.36,clear sky


In [20]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                302
City                   302
Country                298
Lat                    302
Lng                    302
Max Temp               302
Humidity               302
Cloudiness             302
Wind Speed             302
Current Description    302
dtype: int64

In [21]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City_ID                298
City                   298
Country                298
Lat                    298
Lng                    298
Max Temp               298
Humidity               298
Cloudiness             298
Wind Speed             298
Current Description    298
dtype: int64

In [24]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khuzhir,RU,51.55,broken clouds,53.19,107.34,
4,Lebu,CL,54.23,clear sky,-37.62,-73.65,
6,Rikitea,PF,72.88,light rain,-23.12,-134.97,
13,Emba,KZ,61.72,few clouds,48.83,58.14,
14,Albany,US,71.28,scattered clouds,42.60,-73.97,
18,Hilo,US,73.40,broken clouds,19.73,-155.09,
20,Veinticinco De Mayo,AR,68.13,clear sky,-35.43,-60.17,
22,Port Alfred,ZA,59.52,light rain,-33.59,26.89,
23,Chaumont,FR,70.38,scattered clouds,48.12,5.13,
26,Chuy,UY,65.03,clear sky,-33.70,-53.46,


In [25]:
hotel_df.count()

City                   298
Country                298
Max Temp               298
Current Description    298
Lat                    298
Lng                    298
Hotel Name             298
dtype: int64

In [33]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]

    params["location"] = f"{lat},{lng}"
    #print(params)
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        hotel_df.loc[index, "Hotel Name"] = None
        print("Hotel not found... skipping " + hotel_df.loc[index, "City"])   

print("<----------------->")        
print("Updates Complete!")        
print("<----------------->")        

Hotel not found... skipping Emba
Hotel not found... skipping Albany
Hotel not found... skipping Aksarayskiy
Hotel not found... skipping Xudat
Hotel not found... skipping Tiznit
Hotel not found... skipping Druzhba
Hotel not found... skipping Vydrino
Hotel not found... skipping Katangli
Hotel not found... skipping Avera
Hotel not found... skipping Weihe
Hotel not found... skipping Boende
Hotel not found... skipping Ikalamavony
Hotel not found... skipping Shitanjing
Hotel not found... skipping Lodja
Hotel not found... skipping Hare Bay
<----------------->
Updates Complete!
<----------------->


In [34]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khuzhir,RU,51.55,broken clouds,53.19,107.34,Отель на озере Байкал Baikal View
4,Lebu,CL,54.23,clear sky,-37.62,-73.65,Hostal Las Lilas
6,Rikitea,PF,72.88,light rain,-23.12,-134.97,People ThankYou
13,Emba,KZ,61.72,few clouds,48.83,58.14,None
14,Albany,US,71.28,scattered clouds,42.60,-73.97,None
18,Hilo,US,73.40,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
20,Veinticinco De Mayo,AR,68.13,clear sky,-35.43,-60.17,Estancia La Rosada
22,Port Alfred,ZA,59.52,light rain,-33.59,26.89,The Halyards Hotel
23,Chaumont,FR,70.38,scattered clouds,48.12,5.13,Hôtel Restaurant Des Remparts
26,Chuy,UY,65.03,clear sky,-33.70,-53.46,Nuevo Hotel Plaza


In [36]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.info()
hotel_df = hotel_df.dropna()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 298 entries, 0 to 719
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 298 non-null    object 
 1   Country              298 non-null    object 
 2   Max Temp             298 non-null    float64
 3   Current Description  298 non-null    object 
 4   Lat                  298 non-null    float64
 5   Lng                  298 non-null    float64
 6   Hotel Name           283 non-null    object 
dtypes: float64(3), object(4)
memory usage: 26.7+ KB


City                   283
Country                283
Max Temp               283
Current Description    283
Lat                    283
Lng                    283
Hotel Name             283
dtype: int64

In [39]:
hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 283 entries, 0 to 719
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 283 non-null    object 
 1   Country              283 non-null    object 
 2   Max Temp             283 non-null    float64
 3   Current Description  283 non-null    object 
 4   Lat                  283 non-null    float64
 5   Lng                  283 non-null    float64
 6   Hotel Name           283 non-null    object 
dtypes: float64(3), object(4)
memory usage: 17.7+ KB


In [40]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Khuzhir,RU,51.55,broken clouds,53.19,107.34,Отель на озере Байкал Baikal View
4,Lebu,CL,54.23,clear sky,-37.62,-73.65,Hostal Las Lilas
6,Rikitea,PF,72.88,light rain,-23.12,-134.97,People ThankYou
18,Hilo,US,73.40,broken clouds,19.73,-155.09,Hilo Hawaiian Hotel
20,Veinticinco De Mayo,AR,68.13,clear sky,-35.43,-60.17,Estancia La Rosada
22,Port Alfred,ZA,59.52,light rain,-33.59,26.89,The Halyards Hotel
23,Chaumont,FR,70.38,scattered clouds,48.12,5.13,Hôtel Restaurant Des Remparts
26,Chuy,UY,65.03,clear sky,-33.70,-53.46,Nuevo Hotel Plaza
29,Vaini,TO,73.56,few clouds,-21.20,-175.20,Keleti Beach Resort
30,Ancud,CL,52.43,light rain,-41.87,-73.82,Hotel Arena Gruesa


In [42]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [50]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country Code</dt><dd>{Country}</dd>
<dt>Whether Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [51]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))